<a href="https://colab.research.google.com/github/Ghisbe/Alertas_pictogramas/blob/main/Copia_de_Modelo_Bert_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Redes neuronales recurrentes. Transformadores Bert.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
url = "https://drive.google.com/uc?id=1fs7dmydF11O-rw1xX3ZDZd-7dGew73lu"

In [ ]:
import gdown
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
output = 'alertas_lemma.csv'
gdown.download(url, output, quiet=False)

# Leer el archivo CSV en un DataFrame de pandas
alertas = pd.read_csv(output)

Downloading...
From (original): https://drive.google.com/uc?id=1fs7dmydF11O-rw1xX3ZDZd-7dGew73lu
From (redirected): https://drive.google.com/uc?id=1fs7dmydF11O-rw1xX3ZDZd-7dGew73lu&confirm=t&uuid=671bc823-c404-41b0-83c0-904d0d64e75d
To: /content/alertas_lemma.csv
100%|██████████| 143M/143M [00:00<00:00, 143MB/s]


In [ ]:
alertas['event'].unique()

array(['Dust Storm Warning', 'Tropical Storm Warning',
       'Winter Storm Warning', 'Winter Storm Watch',
       'Tropical Storm Watch', 'Storm Warning', 'Storm Watch',
       'Ice Storm Warning'], dtype=object)

In [ ]:
!pip install torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
import torch
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset, RandomSampler, SequentialSampler
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np

# Definir el tokenizador y el modelo
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=8).to(device="cuda:0")

# Tokenizar las descripciones
max_length = 300
alertas['tokenized_description'] = alertas['description'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=max_length, truncation=True))

# Padding de las secuencias
alertas['padded_tokenized_description'] = alertas['tokenized_description'].apply(lambda x: x + [0] * (max_length - len(x)))

# Convertir a tensores
input_ids = torch.tensor(alertas['padded_tokenized_description'].tolist())

# Crear etiquetas
label_dict = {'Dust Storm Warning': 0, 'Tropical Storm Warning': 1, 'Winter Storm Warning': 2,
              'Winter Storm Watch': 3, 'Tropical Storm Watch': 4,'Storm Warning': 5,
              'Storm Watch': 6, 'Ice Storm Warning': 7}
alertas['label'] = alertas['event'].map(label_dict)
labels = torch.tensor(alertas['label'].tolist())

# Dividir en conjunto de entrenamiento y validación
train_inputs, val_inputs, train_labels, val_labels = train_test_split(input_ids, labels, test_size=0.1)

# Crear DataLoaders
batch_size = 16
train_data = TensorDataset(train_inputs, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

val_data = TensorDataset(val_inputs, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

# Crear optimizador y scheduler
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
epochs = 3
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Entrenar el modelo
for epoch in range(epochs):
    model.train()
    for step, batch in enumerate(train_dataloader):
        b_input_ids = batch[0].to(device="cuda:0")
        b_labels = batch[1].to(device="cuda:0")

        optimizer.zero_grad()
        outputs = model(b_input_ids, labels=b_labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()

    # Validación
    model.eval()
    eval_loss = 0
    eval_steps = 0
    predictions, true_labels = [], []
    for batch in val_dataloader:
        b_input_ids = batch[0].to(device="cuda:0")
        b_labels = batch[1].to(device="cuda:0")

        with torch.no_grad():
            outputs = model(b_input_ids, labels=b_labels)
            eval_loss += outputs.loss.item()
            logits = outputs.logits
            predictions.append(torch.argmax(logits, dim=1).cpu().numpy())
            true_labels.append(b_labels.cpu().numpy())
        eval_steps += 1

    avg_val_loss = eval_loss / eval_steps
    predictions = np.concatenate(predictions, axis=0)
    true_labels = np.concatenate(true_labels, axis=0)
    accuracy = accuracy_score(true_labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='weighted')

    print(f"Epoch {epoch + 1} | Validation Loss: {avg_val_loss:.4f} | Accuracy: {accuracy:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f} | F1 Score: {f1:.4f}")

# Evaluación final
model.eval()
predictions, true_labels = [], []
for batch in val_dataloader:
    b_input_ids = batch[0].to(device="cuda:0")
    b_labels = batch[1].to(device="cuda:0")

    with torch.no_grad():
        outputs = model(b_input_ids)
        logits = outputs.logits
        predictions.append(torch.argmax(logits, dim=1).cpu().numpy())
        true_labels.append(b_labels.cpu().numpy())

predictions = np.concatenate(predictions, axis=0)
true_labels = np.concatenate(true_labels, axis=0)
accuracy = accuracy_score(true_labels, predictions)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='weighted')

print(f"Final Evaluation | Accuracy: {accuracy:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f} | F1 Score: {f1:.4f}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


Epoch 1 | Validation Loss: 0.0941 | Accuracy: 0.9620 | Precision: 0.9631 | Recall: 0.9620 | F1 Score: 0.9620
